In [1]:
import csv
import pandas as pd 
import h5py
import os
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import glob
import time

In [3]:
start = time.time()
path = '/home/mvasist/simulations_new/16_params/'

for k in range(607, 1001):   
    if (322<=k<= 336): 
        continue
#     if k> 1 : break

    dfptX= pd.read_csv(path+ 'X_5Msim_'+ str(k) + '.csv', engine ='python', header=None, index_col=0)  #, low_memory = False)
    dfT= pd.read_csv(path+ 'T_5Msim_'+ str(k) + '.csv', engine ='python', header=None, index_col=0)  #, low_memory = False)
    
    ptX = []
    T = []
    ptX.append(dfptX)
    T.append(dfT)
    
    comb_np_array_ptX = np.vstack(ptX)
    ptx = torch.from_numpy(comb_np_array_ptX)
    comb_np_array_T = np.vstack(T)
    th = torch.from_numpy(comb_np_array_T)
    
    x = ptx[:,947*2:]
    indx = (~torch.isnan(x)).sum(axis = 1) == 947
    x = x[indx, :] #deleting rows with nan values
#     p = ptx[:,0:947]
#     t = ptx[:,947:947*2]

    th = th[indx,:] #deleting rows with nan values
    indices = torch.tensor([0,1,2,3,4,8,9,10,11,12,13,14,15])
    th_reduced = torch.index_select(th, 1, indices)
    
    for l in range(int(len(x)/100)):
        print(k, l)
        x_ = x[l*100:(l+1)*100]
        th_reduced_ = th_reduced[l*100:(l+1)*100]
        
        #label 1- dependent 
        xt=[]
        for i in range(len(x_)):
            xt.append(torch.cat([th_reduced_[i],x_[i]]))
        xt = torch.cat(xt)
        xt = xt.reshape(len(x_),int(len(xt)/len(x_)))

        #label 0 - independent
        xt_shuffled=[]
        for j in range(len(x_)):
            r=np.random.randint(0,len(x_))
            if r==j:
                r= np.random.randint(0,len(x_))
            xt_shuffled.append(torch.cat([th_reduced_[j],x_[r]]))
        xt_shuffled = torch.cat(xt_shuffled)
        xt_shuffled = xt_shuffled.reshape(len(x_),int(len(xt_shuffled)/len(x_)))

        #combining them 
        XT = torch.cat((xt,xt_shuffled),axis=0)
        labels = torch.cat((torch.ones(len(x_),1),torch.zeros(len(x_),1)))
        
        # n = 5000
        # low, up = int(len(labels)/2 - (n/2)) , int(len(labels)/2 + (n/2))
        # sample = {'target': labels, 'input': XT }
        # XT_balanced, labels_balanced =  XT[low:up], labels[low:up]

        XT_balanced, labels_balanced =  XT, labels
        sample = {'target': labels_balanced, 'input': XT_balanced }
        XT_2D = XT_balanced.unsqueeze(1)
        
        labels_onehot_dependent = F.pad(labels_balanced[:int(len(labels_balanced)/2)], (0,1,0,0), mode= 'constant', value = 0)
        labels_onehot_independent = F.pad(labels_balanced[int(len(labels_balanced)/2):], (0,1,0,0), mode= 'constant', value = 1)
        labels_onehot= torch.cat((labels_onehot_dependent,labels_onehot_independent), dim = 0)
        #print(labels_onehot.size())
        
        with h5py.File('/home/mvasist/scripts_new/datasets/dataset/_/onehot/13params_vit__'+ str(k)+'_'+str(l)+'.h5', 'w') as f: 
            f.create_dataset('data', data= XT_2D, dtype=np.float32)
            f.create_dataset('label', data=labels_onehot, dtype=np.float32)
    #         f.create_dataset('index', data =  )
            f.close()
    
end = time.time()
print(end-start)   

607 0
607 1
607 2
607 3
607 4
607 5
607 6
607 7
607 8
607 9
607 10
607 11
607 12
607 13
607 14
607 15
607 16
607 17
607 18
607 19
607 20
607 21
607 22
607 23
607 24
607 25
607 26
607 27
607 28
607 29
607 30
607 31
607 32
607 33
607 34
607 35
607 36
607 37
607 38
607 39
607 40
607 41
607 42
607 43
607 44
607 45
607 46
607 47
607 48
607 49
608 0
608 1
608 2
608 3
608 4
608 5
608 6
608 7
608 8
608 9
608 10
608 11
608 12
608 13
608 14
608 15
608 16
608 17
608 18
608 19
608 20
608 21
608 22
608 23
608 24
608 25
608 26
608 27
608 28
608 29
608 30
608 31
608 32
608 33
608 34
608 35
608 36
608 37
608 38
608 39
608 40
608 41
608 42
608 43
608 44
608 45
608 46
608 47
608 48
609 0
609 1
609 2
609 3
609 4
609 5
609 6
609 7
609 8
609 9
609 10
609 11
609 12
609 13
609 14
609 15
609 16
609 17
609 18
609 19
609 20
609 21
609 22
609 23
609 24
609 25
609 26
609 27
609 28
609 29
609 30
609 31
609 32
609 33
609 34
609 35
609 36
609 37
609 38
609 39
609 40
609 41
609 42
609 43
609 44
609 45
609 46
609 47
6

In [7]:
dfT

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,,,,,,,,,,,,,,,,
0,1.235192,-0.587133,0.175632,1.761675,-1.697138,1.637895,10.776192,2.277326,3.727960,0.957259,383.61102,0.844794,0.106771,0.137603,1.563524,0.706490
1,0.442061,0.814650,-3.978606,0.147645,-2.532936,10.294368,12.678186,2.886096,4.597416,1.112396,452.40430,0.431562,0.048114,0.564744,1.167044,0.076044
2,1.106275,0.221644,-2.663154,-0.024220,3.527432,4.556382,11.765339,1.750689,3.356781,1.187492,565.39410,0.910634,0.384538,0.895131,1.683743,0.093300
3,0.920614,0.991139,-4.324566,-0.288490,-3.014508,4.768449,10.946888,1.140078,4.792663,1.552202,699.88510,0.572284,0.398956,0.308650,1.026024,0.287530
4,0.511692,1.149454,2.948245,4.260269,-1.211597,4.310435,9.248787,1.888151,3.728928,1.214105,459.18220,0.157853,0.379502,0.944250,1.361173,0.773017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.245841,0.062328,-3.096630,-1.015651,1.844347,9.225735,9.627146,1.480201,4.921560,1.910057,2038.66980,0.450911,0.990388,0.285080,1.591168,0.514494
96,1.236535,0.751427,-5.902783,3.450962,-1.728772,9.558916,10.595835,1.263204,2.360290,1.908595,1756.66700,0.529897,0.457534,0.846649,1.847322,0.323272
97,1.184359,0.285917,-0.888703,2.709887,-1.028153,6.782586,6.815258,1.514083,4.039507,1.179128,1727.42660,0.623693,0.745883,0.531181,1.107659,0.105320


In [8]:
dfptX

,1,2,3,4,5,6,7,8,9,10,...,2832,2833,2834,2835,2836,2837,2838,2839,2840,2841
0,,,,,,,,,,,,,,,,,,,,,
0,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,2.529022e-23,3.149019e-23,4.153441e-23,4.062050e-23,3.392810e-23,5.043118e-23,5.843860e-23,6.446709e-23,6.545676e-23,5.487162e-23
1,0.000001,0.000001,0.000002,0.000002,0.000002,0.000003,0.000003,0.000004,0.000005,0.000006,...,2.909446e-16,2.772588e-16,2.819114e-16,3.038228e-16,2.425395e-16,2.578080e-16,3.211222e-16,2.713898e-16,2.549147e-16,2.332711e-16
2,0.000001,0.000001,0.000002,0.000002,0.000002,0.000003,0.000003,0.000004,0.000005,0.000006,...,2.785410e-17,2.500638e-17,2.857668e-17,2.725704e-17,2.361853e-17,2.749561e-17,2.985671e-17,2.925637e-17,3.162056e-17,2.666300e-17
3,0.000001,0.000001,0.000002,0.000002,0.000002,0.000003,0.000003,0.000004,0.000005,0.000006,...,5.916667e-17,5.242031e-17,5.854169e-17,5.697816e-17,4.288565e-17,5.532269e-17,6.590138e-17,6.406217e-17,7.261580e-17,5.369450e-17
4,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,3.487631e-22,3.526238e-22,3.896525e-22,3.710948e-22,3.417853e-22,4.205172e-22,4.433497e-22,4.496121e-22,4.551970e-22,4.236836e-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000001,0.000001,0.000002,0.000002,0.000002,0.000003,0.000003,0.000004,0.000005,0.000006,...,5.849204e-15,5.992359e-15,5.943713e-15,6.143019e-15,5.989240e-15,5.914948e-15,5.953030e-15,6.165161e-15,5.792011e-15,5.598261e-15
96,0.000001,0.000001,0.000002,0.000002,0.000002,0.000003,0.000003,0.000004,0.000005,0.000006,...,3.925084e-17,3.452160e-17,4.195843e-17,3.808500e-17,3.138565e-17,3.865216e-17,4.507540e-17,4.414320e-17,5.249962e-17,4.068348e-17
97,0.000001,0.000001,0.000002,0.000002,0.000002,0.000003,0.000003,0.000004,0.000005,0.000006,...,1.340712e-16,1.341837e-16,1.342350e-16,1.343401e-16,1.344359e-16,1.345032e-16,1.345902e-16,1.346889e-16,1.347589e-16,1.348365e-16


In [ ]:
with h5py.File('/home/mvasist/scripts_new/datasets/dataset/_/onehot/13params_vit__1_2.h5', 'r') as f: 
        d = torch.Tensor(f.get('data'))
#         l = torch.Tensor(f.get('label'))
        f.close()


In [1]:
for l in range(int(869/100)):
    print(l)
    

0
1
2
3
4
5
6
7


8